In [416]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [387]:
data = pd.read_csv('data/adult.data', na_values=["?"])
target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [417]:
data = pd.read_csv('data/adult.data', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 12000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = 5000, replace = False)
attack_test_members =  target_dataset.iloc[:7000,:].sample(n = 5000, replace = False)

In [ ]:
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"])
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"])
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_28K.csv', na_values=["?"])
attack_test_members = pd.read_csv('data/adultODS1_to_3K.csv', na_values=["?"])

In [418]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [419]:
def old_transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column
    

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data
            print(x.iloc[:,j])

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [420]:
def transform_data(dataset):
    
    for col in [1,3,5,6,7,8,9,13,14]:
        le = LabelEncoder()
        dataset[col] = le.fit_transform(dataset[col].astype('str'))
    
    # normalize the values
    x_range = [i for i in range(14)]
    dataset[x_range] = dataset[x_range]/dataset[x_range].max()

    x = dataset[x_range].values
    y = dataset[14].values
    
    dim = x.shape[1]
    
    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [421]:
def load_target_data(dataset, test_ratio):
    x, y, dim = transform_data(dataset)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:7000,]
    testX = x[7000:,]
    trainY = y[0:7000,]
    testY = y[7000:,]
    return (trainX, trainY), (testX, testY), dim

In [422]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [423]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio):
    x, y, _ = transform_data(dataset)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_adult{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [424]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_adult{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_simple_mlp(n_class,dim, channel)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model

In [425]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [426]:
def attack_mlp(pix,d):

    model = Sequential()
    model.add(Dense(64, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.1))

#     model.add(Dense(32))
#     model.add(Activation("tanh"))
#     model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=1
    
    return model, act_layer

In [427]:
def prep_attack_data(n_attack_data):

    attack_mem = pd.DataFrame(n_attack_data[0][0])
    attack_nmem = pd.DataFrame(n_attack_data[0][1])
    
    attack_mem_status = pd.DataFrame(n_attack_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(n_attack_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(n_attack_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(n_attack_data[2][1])
    real_class_nmem.columns = ["y"]

    memdf = pd.concat([attack_mem,attack_nmem],axis=0)
    memdf = memdf.reset_index(drop=True)

    memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
    memstatus = memstatus.reset_index(drop=True)

    realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
    realclass = realclass.reset_index(drop=True)

    attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return attack_df

In [428]:
def prep_validation_data(attack_test_data):

    attack_mem = pd.DataFrame(attack_test_data[0][0])
    attack_nmem = pd.DataFrame(attack_test_data[0][1])
    
    attack_mem_status = pd.DataFrame(attack_test_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(attack_test_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(attack_test_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(attack_test_data[2][1])
    real_class_nmem.columns = ["y"]
    
    mem_df = pd.concat([attack_mem,real_class_mem],axis=1)
    nmem_df = pd.concat([attack_nmem,real_class_nmem],axis=1)

#     memdf = pd.concat([attack_mem,attack_nmem],axis=0)
#     memdf = memdf.reset_index(drop=True)

#     memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
#     memstatus = memstatus.reset_index(drop=True)

#     realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
#     realclass = realclass.reset_index(drop=True)

#     attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return mem_df, nmem_df

In [429]:
def load_attack_test_data(members, nonmembers):
    memberX, memberY, _ = transform_data(members)
    
    nonmemberX, nonmemberY, _ = transform_data(nonmembers)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [430]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [431]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        
        d=1
        pix = full_cx_data.shape[1]
        classifier, _ = attack_mlp(pix,d)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        pred_y = classifier.predict(checkmem_prediction_vals[c_indices])
        print(pred_y)
        c_pred_y = np.argmax(pred_y, axis=1)
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall

In [432]:
def shokri_attack(attack_df, mem_validation, nmem_validation):
    
    predicted_membership, predicted_nmembership, true_membership, TP_idx, TN_idx  = [], [], [], [], []

    class_val = np.unique(attack_df['y'])
    ncval=attack_df.shape[1]-1
    
    for c_val in class_val:

        print(c_val)
        
        filter_rec_all = attack_df[(attack_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        attack_feat = filter_rec_all.iloc[:, 0:ncval]
        attack_class = filter_rec_all['membership']
             
        d=1
        pix = attack_feat.shape[1]
        
        attack_model, _ = attack_mlp(pix,d)
        
       
        history = attack_model.fit(attack_feat, attack_class, epochs=EPS, batch_size=32, verbose=0)
        
        mcval=mem_validation.shape[1]-1
        
        
        check_mem_feat = mem_validation[mem_validation['y']==c_val]
        check_nmem_feat = nmem_validation[nmem_validation['y']==c_val]
        
        if (len(check_mem_feat)!=0) and (len(check_nmem_feat)!=0):
        
            check_mem_feat_idx =  np.array(check_mem_feat.index)


            check_nmem_feat_idx =  np.array(check_nmem_feat.index)

            #print(check_nmem_feat_idx)
            #print(np.argmax(mpred,axis=1)==0)


            mpred = attack_model.predict(np.array(check_mem_feat))    
            predicted_membership.append(np.argmax(mpred,axis=1) )

            nmpred = attack_model.predict(np.array(check_nmem_feat))    
            predicted_nmembership.append(np.argmax(nmpred,axis=1) )        



            TP_idx.append(check_mem_feat_idx[np.where(np.argmax(mpred,axis=1)==1)[0]])

            TN_idx.append(check_nmem_feat_idx[np.where(np.argmax(nmpred,axis=1)==0)[0]])

    pred_members = np.array([item for sublist in predicted_membership for item in sublist])
    pred_nonmembers = np.array([item for sublist in predicted_nmembership for item in sublist])
    
    TP_idx_list = np.array([item for sublist in TP_idx for item in sublist])
    TN_idx_list = np.array([item for sublist in TN_idx for item in sublist])
    
    members=np.array(list(pred_members))
    nonmembers=np.array(list(pred_nonmembers))
    
    pred_membership = np.concatenate([members,nonmembers])
    ori_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    return pred_membership, ori_membership, TP_idx_list, TN_idx_list

In [433]:
# trian target model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

(target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio)
target_model,_ = build_simple_mlp(n_class,dim, channel)
#get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
print('\n', 'Model test accuracy:', score[1])


 Model test accuracy: 0.840666651725769


In [434]:
# prepare attack test data

members = []
nonmembers = []

memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers)

# member
target_model_member_pred = target_model.predict(memberX, batch_size=32)
target_model_member_class = np.argmax(memberY, axis=1)
target_model_member_pred = np.vstack(target_model_member_pred)
#target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
members.append(np.ones(len(target_model_member_pred)))
members = [item for sublist in members for item in sublist]


# nonmember
target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
#target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
nonmembers = [item for sublist in nonmembers for item in sublist]

full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

print('\n pred', full_attack_test_pred_val)
print('\n class', full_attack_test_class_status)
print('\n mem status', full_attack_test_mem_status)

attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)

157/157 [==============================] - 0s 2ms/step

 pred (array([[9.9986386e-01, 1.3606111e-04],
       [9.9997908e-01, 2.0879399e-05],
       [2.3124990e-01, 7.6875007e-01],
       ...,
       [4.2168081e-01, 5.7831919e-01],
       [9.9958807e-01, 4.1195491e-04],
       [9.9999887e-01, 1.0409306e-06]], dtype=float32), array([[0.99833614, 0.00166391],
       [0.33577356, 0.66422635],
       [0.9685055 , 0.03149452],
       ...,
       [0.70970327, 0.29029676],
       [0.9149672 , 0.08503288],
       [0.9914294 , 0.00857064]], dtype=float32))

 class (array([0, 0, 0, ..., 1, 0, 0]), array([0, 1, 0, ..., 0, 0, 0]))

 mem status (array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([0, 0, 0, ..., 0, 0, 0], dtype=int32))


In [435]:
# save attack test dataset to csv
df = pd.DataFrame()
mem = pd.Series(full_attack_test_pred_val[0][:,0])
nonmem = pd.Series(full_attack_test_pred_val[1][:,0])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
len(total)
df['prob_class_0'] = total
df = df.reset_index()
mem = pd.Series(full_attack_test_pred_val[0][:,1])
nonmem = pd.Series(full_attack_test_pred_val[1][:,1])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['prob_class 1'] = total

mem = pd.Series(full_attack_test_mem_status[0][:])
nonmem = pd.Series(full_attack_test_mem_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['mem_status'] = total

mem = pd.Series(full_attack_test_class_status[0][:])
nonmem = pd.Series(full_attack_test_class_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['class_status'] = total

df.drop("index", axis=1, inplace=True)


df.to_csv('attack_test_data.csv')

In [436]:
#prepare shadow dataset
n_shadow_models = 20
shadow_data_size = 5000
test_ratio = 0.3

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio)

shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_testX =  1500 shadow_i_testY =  1500
shadow_i_trainX =  3500 shadow_i_trainY =  3500 shadow_i_tes

In [437]:
#train shadow model
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = train_shadow_models(n_shadow_models, n_class, dim, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  89.99999761581421 Shadow Test acc :  84.46666598320007
47/47 [==============================] - 0s 1ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  88.88571262359619 Shadow Test acc :  83.60000252723694
47/47 [==============================] - 0s 1ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  90.34285545349121 Shadow Test acc :  82.99999833106995
47/47 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  88.17142844200134 Shadow Test acc :  82.13333487510681
47/47 [==============================] - 0s 1ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  90.2571439743042 Shadow Test acc :  83.33333134651184
47/47 [==============================] - 0s 1ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  89.7428572177887 Shadow Test acc :  82.40000009536743
47/47 [==============================] - 0s 1ms

In [438]:
n_attack_data

((array([[3.8187438e-01, 6.1812568e-01],
         [9.7239673e-01, 2.7603187e-02],
         [4.6574762e-01, 5.3425235e-01],
         ...,
         [9.9999976e-01, 2.9060186e-07],
         [9.9992776e-01, 7.2289426e-05],
         [1.3614281e-01, 8.6385721e-01]], dtype=float32),
  array([[1.7935550e-01, 8.2064456e-01],
         [9.9998313e-01, 1.6831158e-05],
         [1.2823899e-01, 8.7176102e-01],
         ...,
         [1.1326178e-01, 8.8673818e-01],
         [4.1509447e-01, 5.8490551e-01],
         [9.9922812e-01, 7.7187520e-04]], dtype=float32)),
 (array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int32)),
 (array([1, 0, 1, ..., 0, 0, 1]), array([1, 0, 1, ..., 1, 1, 0])))

In [439]:
attack_df = prep_attack_data(n_attack_data)
attack_df

,0,1,y,membership
0,0.381874,0.618126,1,1
1,0.972397,0.027603,0,1
2,0.465748,0.534252,1,1
3,0.871952,0.128048,0,1
4,0.350740,0.649260,1,1
...,...,...,...,...
99995,0.999985,0.000015,0,0
99996,0.997915,0.002085,0,0
99997,0.113262,0.886738,1,0
99998,0.415094,0.584906,1,0


In [440]:
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
nmem_validation

,0,1,y
0,0.998336,0.001664,0
1,0.335774,0.664226,1
2,0.968506,0.031495,0
3,0.005760,0.994240,0
4,0.716497,0.283503,0
...,...,...,...
4995,0.987254,0.012746,0
4996,0.520258,0.479742,0
4997,0.709703,0.290297,0
4998,0.914967,0.085033,0


In [443]:
tp, fp, fn, tn, precision, advj, acc, recall = train_attack_model(n_attack_data, attack_test_data)

Class :  0
237/237 [==============================] - 0s 1ms/step
[[0.2934423  0.70655763]
 [0.29352674 0.7064733 ]
 [0.4228407  0.5771593 ]
 ...
 [0.29022825 0.70977175]
 [0.28037447 0.7196255 ]
 [0.2873     0.71270007]]
Class :  1
76/76 [==============================] - 0s 1ms/step
[[0.2828035  0.7171965 ]
 [0.31496772 0.6850323 ]
 [0.2962644  0.7037356 ]
 ...
 [0.28893104 0.7110689 ]
 [0.269594   0.730406  ]
 [0.28169298 0.718307  ]]
TP: 4963     FP: 4834     FN: 37     TN: 166
PPV: 0.5066
Advantage: 0.0258


In [441]:
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_df, mem_validation, nmem_validation)

0
119/119 [==============================] - 0s 1ms/step
1
38/38 [==============================] - 0s 1ms/step


In [442]:
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)

TP: 4963     FP: 4830     FN: 37     TN: 170
PPV: 0.5068
Advantage: 0.0266


In [282]:
#save the prepared attack data on disk
np.savez(DATA_PATH + 'attack_purchase_data.npz', n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init)

/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


INFO:tensorflow:Assets written to: ram://a9382bc7-fd48-42a6-8436-af2bd2196281/assets


In [ ]:
# read stored attack model training data 
data_name = 'attack_adult_data.npz'
with np.load(DATA_PATH + data_name, allow_pickle=True) as f:
        n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = [f['arr_%d' % i] for i in range(len(f.files))]

In [283]:
tp, fp, fn, tn, precision, advj, acc, recall, target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)
#target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)

Class :  0
5/5 [==============================] - 0s 2ms/step
[[0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 0.49993768]
 [0.50006235 

4/4 [==============================] - 0s 2ms/step
[[0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.49783623 0.5021638 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.49819902 0.50180095]
 [0.49803716 0.5019628 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.4978264  0.5021736 ]
 [0.49785376 0.5021462 ]
 [0.4978264  0.5021736 ]

6/6 [==============================] - 0s 2ms/step
[[0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]
 [0.5003825  0.49961737]

7/7 [==============================] - 0s 1ms/step
[[0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]
 [0.49988297 0.500117  ]

6/6 [==============================] - 0s 2ms/step
[[0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]
 [0.4999905  0.50000954]

...

6/6 [==============================] - 0s 2ms/step
[[0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]
 [0.4999184  0.50008154]

6/6 [==============================] - 0s 1ms/step
[[0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]
 [0.49985263 0.50014734]

6/6 [==============================] - 0s 1ms/step
[[0.49137872 0.5086213 ]
 [0.49141365 0.50858635]
 [0.49137807 0.50862193]
 [0.4916091  0.5083909 ]
 [0.4913935  0.50860655]
 [0.49164954 0.5083505 ]
 [0.491782   0.5082179 ]
 [0.4915418  0.50845826]
 [0.4913939  0.5086061 ]
 [0.49141568 0.50858426]
 [0.49137592 0.5086241 ]
 [0.4913947  0.5086052 ]
 [0.49138346 0.5086165 ]
 [0.49234447 0.5076555 ]
 [0.49193835 0.50806165]
 [0.49144372 0.5085562 ]
 [0.4914999  0.50850004]
 [0.4913765  0.5086235 ]
 [0.4913758  0.50862426]
 [0.49138695 0.50861305]
 [0.49138457 0.5086154 ]
 [0.49143812 0.5085619 ]
 [0.4914008  0.5085993 ]
 [0.49139115 0.50860876]
 [0.49138042 0.5086195 ]
 [0.49138227 0.5086177 ]
 [0.49264032 0.5073598 ]
 [0.49166438 0.5083356 ]
 [0.49145928 0.5085408 ]
 [0.49140564 0.5085943 ]
 [0.49137697 0.50862294]
 [0.491379   0.50862104]
 [0.4915921  0.5084079 ]
 [0.491377   0.50862294]
 [0.49413735 0.50586265]
 [0.49138436 0.5086156 ]
 [0.49153295 0.508467  ]
 [0.4916103  0.5083897 ]

3/3 [==============================] - 0s 2ms/step
[[0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]
 [0.49995297 0.500047  ]

7/7 [==============================] - 0s 1ms/step
[[0.49391168 0.5060883 ]
 [0.49553898 0.5044611 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49637294 0.503627  ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.4939773  0.5060227 ]
 [0.49391168 0.5060883 ]
 [0.49528372 0.5047162 ]
 [0.49391168 0.5060883 ]
 [0.49393165 0.5060684 ]
 [0.49409813 0.5059018 ]
 [0.49412006 0.50588   ]
 [0.49407634 0.50592357]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49414244 0.5058575 ]
 [0.49426708 0.5057329 ]
 [0.49441642 0.5055835 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.4939833  0.5060167 ]
 [0.49391168 0.5060883 ]
 [0.4948462  0.50515383]
 [0.49391168 0.5060883 ]
 [0.49391168 0.5060883 ]
 [0.49431562 0.50568444]
 [0.49391168 0.5060883 ]
 [0.49393082 0.5060692 ]
 [0.49408615 0.5059139 ]

In [284]:
precision

0.5193872315473136

In [ ]:
target_membership

In [ ]:
predicted_membership

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership)

In [ ]:
df

In [ ]:
df.to_csv('test.csv')

In [ ]:
attack_data = n_attack_data
check_membership = attack_test_data
n_hidden=50
learning_rate=0.01
batch_size=200
epochs=50
model='nn'
l2_ratio=1e-7

In [ ]:
x, y,  classes = attack_data

train_x = x[0]
train_y = y[0]
test_x = x[1]
test_y = y[1]
train_classes = classes[0]
test_classes = classes[1]

#print(tra)


In [ ]:
checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership

checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])

train_indices = np.arange(len(train_x))
test_indices = np.arange(len(test_x))
unique_classes = np.unique(train_classes)


predicted_membership, target_membership = [], []

In [ ]:
for c in unique_classes:
    print("Class : ", c)
    c_train_indices = train_indices[train_classes == c]
    c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
    c_test_indices = test_indices[test_classes == c]
    c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
    c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)  

    full_cx_data=(c_train_x,c_test_x)
    full_cx_data = np.vstack(full_cx_data)

    full_cy_data=(c_train_y,c_test_y)
    full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
    full_cy_data = to_categorical(full_cy_data)

    classifier = define_attack_model(2)
    history = classifier.fit(full_cx_data, full_cy_data, epochs=200, batch_size=32, verbose=0)
    #classifier.save('model/attack_model_class{}'.format(c))

    #get predictions on real train and test data
    c_indices = np.where(checkmem_class_status==c)
    predict_y = classifier.predict(checkmem_prediction_vals[c_indices])
    print(predict_y)
    c_pred_y = np.argmax(classifier.predict(checkmem_prediction_vals[c_indices]),axis=1)
    #c_pred_y = np.where(predict_y > 0.5, 1,0)
    #c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])

    c_target_y = checkmem_membership_status[c_indices]


    target_membership.append(c_target_y)
    predicted_membership.append(c_pred_y)




In [ ]:
c_indices = np.where(checkmem_class_status==c)
predict_y = classifier.predict(checkmem_prediction_vals[c_indices], batch_size=32)
c_pred_y = np.where(predict_y > 0.5, 1,0)
c_target_y = checkmem_membership_status[c_indices]
target_membership.append(c_target_y)
predicted_membership.append(c_pred_y)

In [ ]:
target_membership=np.array([item for sublist in target_membership for item in sublist])
predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])

In [ ]:
prety_print_result (target_membership,predicted_membership)

In [ ]:
np.vstack(predicted_membership).shape

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership.reshape((len(predicted_membership))))

In [ ]:
df

In [ ]:
predicted_membership